#Data warehouse OLAP queries using Spark-SQL

In [3]:
# install py-spark
!pip install --quiet pyspark

In [6]:
# mount Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
# Start Spark Session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("BDP Project") \
    .getOrCreate()

In [8]:
# A function to create table from a CSV
def csvToTable(file_name, table_name):
  db_name = ""
  #table_name = db_name + table_name
  df = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep", ",") \
  .csv( "gdrive/My Drive/Dataset/" + file_name )
  df.createOrReplaceTempView( table_name +"df" ) 

  tokyo_sql = "drop table IF EXISTS " + table_name
  spark.sql( tokyo_sql )

  tokyo_sql = "create table " + table_name \
  + " USING PARQUET AS" \
  + " SELECT * FROM " + table_name + "df"
  spark.sql( tokyo_sql )

  tokyo_sql = "describe " + table_name
  print("Table " + table_name + "Done Successfully!")
  spark.sql( tokyo_sql ).show()

  spark.catalog.dropTempView( table_name + "df" )

In [9]:
# Create Tables
csvToTable("Athletes.csv", "Athletes")
csvToTable("Coaches.csv", "Coaches")
csvToTable("EntriesGender.csv", "Gender")
csvToTable("Teams.csv", "Teams")
csvToTable("Medals.csv", "Medals")

Table AthletesDone Successfully!
+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|      Name|   string|   null|
|   Country|   string|   null|
|Discipline|   string|   null|
+----------+---------+-------+

Table CoachesDone Successfully!
+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|      Name|   string|   null|
|   Country|   string|   null|
|Discipline|   string|   null|
|     Event|   string|   null|
+----------+---------+-------+

Table GenderDone Successfully!
+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|Discipline|   string|   null|
|    Female|      int|   null|
|      Male|      int|   null|
|     Total|      int|   null|
+----------+---------+-------+

Table TeamsDone Successfully!
+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|      Name|   string|   null|
|Discipline|   string|   null|
|  

In [10]:
#show Tables
tables = "SHOW TABLES"
spark.sql( tables ).show()

#you can also use following
#spark.catalog.listTables()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default| athletes|      false|
|  default|  coaches|      false|
|  default|   gender|      false|
|  default|   medals|      false|
|  default|    teams|      false|
+---------+---------+-----------+



showing contents of tables using"select" statement

In [11]:
athletes = "select * from athletes limit 10"
spark.sql( athletes ).show()

+-----------------+--------------------+-------------------+
|             Name|             Country|         Discipline|
+-----------------+--------------------+-------------------+
|  AALERUD Katrine|              Norway|       Cycling Road|
|      ABAD Nestor|               Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|               Italy|             Rowing|
|   ABALDE Alberto|               Spain|         Basketball|
|    ABALDE Tamara|               Spain|         Basketball|
|        ABALO Luc|              France|           Handball|
|     ABAROA Cesar|               Chile|             Rowing|
|    ABASS Abobakr|               Sudan|           Swimming|
| ABBASALI Hamideh|Islamic Republic ...|             Karate|
|    ABBASOV Islam|          Azerbaijan|          Wrestling|
+-----------------+--------------------+-------------------+



In [12]:
spark.sql(athletes).printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [13]:
coaches = "select * from coaches limit 10"
spark.sql( coaches ).show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       Côte d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
+--------------------+--------------------+-----------------+--------+



In [14]:
spark.sql(coaches).printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [15]:
gender = "select * from gender limit 10"
spark.sql( gender ).show()

+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
|          Badminton|    86|  87|  173|
|  Baseball/Softball|    90| 144|  234|
|         Basketball|   144| 144|  288|
|   Beach Volleyball|    48|  48|   96|
|             Boxing|   102| 187|  289|
+-------------------+------+----+-----+



In [16]:
spark.sql(gender).printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [17]:
medals = "select * from medals limit 10"
spark.sql( medals ).show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|             Country|Gold|Silver|Bronze|Total|Rank_by_Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
+----+--------------------+----+------+------+-----+-------------+



In [18]:
spark.sql(medals).printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank_by_Total: integer (nullable = true)



In [19]:
teams = "select * from teams limit 10"
spark.sql( teams ).show()

+-----------+--------------+--------------------+-----+
|       Name|    Discipline|             Country|Event|
+-----------+--------------+--------------------+-----+
|    Belgium|3x3 Basketball|             Belgium|  Men|
|      China|3x3 Basketball|People's Republic...|  Men|
|      China|3x3 Basketball|People's Republic...|Women|
|     France|3x3 Basketball|              France|Women|
|      Italy|3x3 Basketball|               Italy|Women|
|      Japan|3x3 Basketball|               Japan|  Men|
|      Japan|3x3 Basketball|               Japan|Women|
|     Latvia|3x3 Basketball|              Latvia|  Men|
|   Mongolia|3x3 Basketball|            Mongolia|Women|
|Netherlands|3x3 Basketball|         Netherlands|  Men|
+-----------+--------------+--------------------+-----+



In [20]:
spark.sql(teams).printSchema()

root
 |-- Name: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



#Queries

In [21]:
sql="SELECT COUNT(DISTINCT discipline) AS DISTINCT_SPORT FROM athletes"
spark.sql( sql ).show()

+--------------+
|DISTINCT_SPORT|
+--------------+
|            46|
+--------------+



In [22]:
sql1 = "Select Name,Country, Discipline AS SPORT FROM athletes GROUP BY CUBE (Name,Country,Discipline)"
spark.sql( sql1 ).show()

+--------------------+------------+-------------------+
|                Name|     Country|              SPORT|
+--------------------+------------+-------------------+
|                null|        null|Rhythmic Gymnastics|
|    ABDELWAHED Ahmed|        null|               null|
|       ABEL Jennifer|        null|               null|
|      ACOSTA Marcelo| El Salvador|           Swimming|
|                null|        null|       Canoe Slalom|
|ADEKUOROYE Odunay...|     Nigeria|               null|
|    AKINYI Elizabeth|        null|             Boxing|
|   ALDABBAGH Yasmeen|Saudi Arabia|          Athletics|
|ALEMANNO Maria Gr...|        null|               null|
|                null|South Africa|          Athletics|
|     ALLIKVEE Martin|     Estonia|               null|
|    ALTUHAYFAN Hamad|        null|           Football|
|    ALVARADO Luciana|  Costa Rica|Artistic Gymnastics|
| ANASTASAKIS Michail|        null|          Athletics|
|   ANDERSON Doneisha|     Bahamas|             

In [23]:
sql2 = "SELECT SUM(Gold) AS Total_Gold, SUM(Silver) AS Total_Silver, SUM(Bronze) AS Total_Bronze FROM Medals "
spark.sql( sql2 ).show()

+----------+------------+------------+
|Total_Gold|Total_Silver|Total_Bronze|
+----------+------------+------------+
|       340|         338|         402|
+----------+------------+------------+



In [24]:
#Finding Top 10 Countries according to World ranking in Tokyo Olympics
sql3 = "Select * From medals ORDER BY Rank_by_Total LIMIT 10"
spark.sql( sql3 ).show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|             Country|Gold|Silver|Bronze|Total|Rank_by_Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   3|               Japan|  27|    14|    17|   58|            5|
|   6|           Australia|  17|     7|    22|   46|            6|
|  10|               Italy|  10|    10|    20|   40|            7|
|   9|             Germany|  10|    11|    16|   37|            8|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
+----+--------------------+----+------+------+-----+-------------+



In [25]:
sql4="Select Discipline, Total AS Participants FROM gender Group BY CUBE (Discipline,Participants) ORDER BY (Discipline,Participants) desc"
spark.sql( sql4 ).show()

+--------------------+------------+
|          Discipline|Participants|
+--------------------+------------+
|           Wrestling|         289|
|           Wrestling|        null|
|       Weightlifting|         197|
|       Weightlifting|        null|
|          Water Polo|         268|
|          Water Polo|        null|
|          Volleyball|         288|
|          Volleyball|        null|
|           Triathlon|         110|
|           Triathlon|        null|
|Trampoline Gymnas...|          32|
|Trampoline Gymnas...|        null|
|              Tennis|         191|
|              Tennis|        null|
|           Taekwondo|         130|
|           Taekwondo|        null|
|        Table Tennis|         172|
|        Table Tennis|        null|
|            Swimming|         779|
|            Swimming|        null|
+--------------------+------------+
only showing top 20 rows



In [26]:
sql5 = "Select * FROM medals WHERE Rank_by_Total = 25"
spark.sql( sql5 ).show()

+----+-------+----+------+------+-----+-------------+
|Rank|Country|Gold|Silver|Bronze|Total|Rank_by_Total|
+----+-------+----+------+------+-----+-------------+
|  19|  Kenya|   4|     4|     2|   10|           25|
+----+-------+----+------+------+-----+-------------+



In [27]:
sql6 = "Select * FROM medals Where Country = 'India'"
spark.sql( sql6 ).show()

+----+-------+----+------+------+-----+-------------+
|Rank|Country|Gold|Silver|Bronze|Total|Rank_by_Total|
+----+-------+----+------+------+-----+-------------+
|  48|  India|   1|     2|     4|    7|           33|
+----+-------+----+------+------+-----+-------------+



In [28]:
sql7 = "select Distinct Discipline from athletes "
spark.sql(sql7).show()

+--------------------+
|          Discipline|
+--------------------+
|              Tennis|
|              Boxing|
|   Marathon Swimming|
|                Golf|
|              Rowing|
|   Baseball/Softball|
|                Judo|
|             Sailing|
|            Swimming|
|Cycling BMX Frees...|
|          Basketball|
|            Handball|
| Rhythmic Gymnastics|
|              Karate|
|           Triathlon|
|           Badminton|
|        Canoe Sprint|
|           Athletics|
|       Cycling Track|
|    Beach Volleyball|
+--------------------+
only showing top 20 rows



In [29]:
sql8 = "select count(Distinct Country) from athletes "
spark.sql(sql8).show()

+-----------------------+
|count(DISTINCT Country)|
+-----------------------+
|                    206|
+-----------------------+



In [30]:
sql9 = "Select * FROM medals Where total >= 45"
spark.sql( sql9 ).show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|             Country|Gold|Silver|Bronze|Total|Rank_by_Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
+----+--------------------+----+------+------+-----+-------------+



In [31]:
##  Query: - Particpants at across countries

sql10='''
select * from (
select count(name) as Count_of_Players,Country
from athletes
group by Country) a
order by a.count_of_players desc
'''
spark.sql( sql10 ).show()

+----------------+--------------------+
|Count_of_Players|             Country|
+----------------+--------------------+
|             615|United States of ...|
|             586|               Japan|
|             470|           Australia|
|             401|People's Republic...|
|             400|             Germany|
|             377|              France|
|             368|              Canada|
|             366|       Great Britain|
|             356|               Italy|
|             324|               Spain|
|             318|                 ROC|
|             291|              Brazil|
|             274|         Netherlands|
|             223|   Republic of Korea|
|             202|         New Zealand|
|             195|              Poland|
|             180|           Argentina|
|             171|        South Africa|
|             155|              Mexico|
|             155|             Hungary|
+----------------+--------------------+
only showing top 20 rows



In [32]:
sql11='''
select * from (
select count(name) as Count_of_Coaches,Country from coaches
group by Country) a
order by a.Count_of_Coaches desc
'''
spark.sql( sql11 ).show()

+----------------+--------------------+
|Count_of_Coaches|             Country|
+----------------+--------------------+
|              35|               Japan|
|              28|United States of ...|
|              28|               Spain|
|              22|           Australia|
|              16|              Canada|
|              14|               Italy|
|              12|People's Republic...|
|              12|                 ROC|
|              12|        South Africa|
|              12|               Egypt|
|              11|           Argentina|
|              10|              France|
|              10|           Venezuela|
|              10|         Netherlands|
|               9|             Nigeria|
|               9|             Germany|
|               8|         New Zealand|
|               8|              Mexico|
|               7|       Great Britain|
|               7|   Republic of Korea|
+----------------+--------------------+
only showing top 20 rows



In [33]:
##  Query: - Coaches vs Player Ratio

sql12='''
select
coach_table.Country,
Count_of_Players,
Count_of_Coaches,
round(Count_of_Players/Count_of_Coaches,2) as Player_Coach_Ratio

from 
(select * from (
select count(name) as Count_of_Players,Country
from athletes
group by Country) a
order by a.count_of_players desc) player_table

join (
select * from (
select count(name) as Count_of_Coaches,Country from coaches
group by Country) a
order by a.Count_of_Coaches desc) coach_table

on player_table.Country=coach_table.Country

''';
spark.sql( sql12 ).show()

+--------------------+----------------+----------------+------------------+
|             Country|Count_of_Players|Count_of_Coaches|Player_Coach_Ratio|
+--------------------+----------------+----------------+------------------+
|       Côte d'Ivoire|              29|               5|               5.8|
|Islamic Republic ...|              66|               2|              33.0|
|              Sweden|             129|               3|              43.0|
|   Republic of Korea|             223|               7|             31.86|
|                Fiji|              28|               2|              14.0|
|              Turkey|             102|               1|             102.0|
|             Germany|             400|               9|             44.44|
|            Cambodia|               3|               1|               3.0|
|              France|             377|              10|              37.7|
|              Greece|              75|               6|              12.5|
|           

In [34]:
##  Query: -  Teams vs Disciplines
sql13='''
select * from 
(select Name,
count(distinct discipline) as Count_of_Discipline
from teams
group by name) a
order by a.Count_of_Discipline desc
''';
spark.sql(sql13).show()

+-----------------+-------------------+
|             Name|Count_of_Discipline|
+-----------------+-------------------+
|            Japan|                 19|
|    United States|                 17|
|            China|                 16|
|           France|                 15|
|           Canada|                 15|
|        Australia|                 14|
|            Italy|                 14|
|              ROC|                 13|
|          Germany|                 12|
|            Spain|                 12|
|      Netherlands|                 12|
|Republic of Korea|                 10|
|    Great Britain|                 10|
|           Brazil|                 10|
|           Poland|                  8|
|     South Africa|                  8|
|            Egypt|                  7|
|          Belgium|                  7|
|          Ukraine|                  7|
|        Argentina|                  6|
+-----------------+-------------------+
only showing top 20 rows

